In [ ]:
# Importing libraries

import numpy as np
import pandas as pd

In [ ]:
# Importing and Reading data

ds=pd.read_csv("new_cleaned_data.csv")
ds = ds.loc[:, ~ds.columns.str.contains('^Unnamed')]                # removing unnamed column
ds

In [ ]:
ds.info()

In [ ]:
# Splitting data into independent and dependent variables

X=ds[['company','name','year','kms_driven','fuel_type']]
y=ds[['Price']]

In [ ]:
print(len(ds['company'].unique()))
print(len(ds['name'].unique()))
print(len(ds['fuel_type'].unique()))


In [ ]:
# implement OneHotEncoder

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
ohe=OneHotEncoder()
ohe.fit(X[["company","name","fuel_type"]])

In [ ]:
ohe.categories_

In [ ]:
ct = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore',categories=ohe.categories_), ["company","name","fuel_type"]),remainder='passthrough',force_int_remainder_cols=False,sparse_threshold = 0)
ct

In [ ]:
# making pipeline

from sklearn.linear_model import LinearRegression
reg=LinearRegression()

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(ct,reg)
pipe

In [ ]:
# Splitting dataset into training and testing

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
scores = []
for i in range(0,101):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.10,random_state= i)
    pipe.fit(X_train,y_train)
    result=pipe.predict(X_test)
    score = r2_score(y_test,result)
    scores.append(score)

In [ ]:
# Finding best value

bestindex = np.argmax(scores)
scores[bestindex]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.10,random_state=bestindex)
pipe.fit(X_train,y_train)

In [ ]:
# predict for user_input

company=input("enter company name : ")
name=input("enter car name : ")
year=int(input("enter year : "))
kms_driven=int(input("enter kms driven : "))
fuel_type=input("enter fuel type : ")
columns=["company","name","year","kms_driven","fuel_type"]
myinput=pd.DataFrame(columns=columns,data=[[company,name,year,kms_driven,fuel_type]])
result=pipe.predict(myinput)
print("You should buy it for ~ price : ",abs(round(result[0,0])))

enter company name :  Honda
enter car name :  Amaze
enter year :  2013
enter kms driven :  284999
enter fuel type :  46000


In [ ]:
import pickle as pkl